# Tensorflow Project Exercise

We'll use the [Bank Authentication Data Set](https://archive.ics.uci.edu/ml/datasets/banknote+authentication) from the UCI repository.

The data consists of 5 columns:

* variance of Wavelet Transformed image (continuous)
* skewness of Wavelet Transformed image (continuous)
* curtosis of Wavelet Transformed image (continuous)
* entropy of image (continuous)
* class (integer)

Where class indicates whether or not a Bank Note was authentic.

This sort of task is perfectly suited for Neural Networks and Deep Learning! Just follow the instructions below to get started!

## Get the Data

*Use pandas to read in the data_banknote_authentication.txt file*

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
sns.set()

In [ ]:
data = pd.read_csv('data_banknote_authentication.txt')
data

*Check the head of the Data*

In [ ]:
data.head()
data['Class']

## EDA

We'll just do a few quick plots of the data.

*Import seaborn and set matplolib inline for viewing*

In [ ]:
import matplotlib.pyplot as plt

*Create a Countplot of the Classes (Authentic 1 vs Fake 0)*

In [ ]:
sns.countplot(data['Class'])

*Create a PairPlot of the Data with Seaborn, set Hue to Class*

In [ ]:
sns.pairplot(data, hue='Class')

## Data Preparation 

When using Neural Network and Deep Learning based systems, it is usually a good idea to Standardize your data, this step isn't actually necessary for our particular data set, but let's run through it for practice!

### Standard Scaling



In [ ]:
from sklearn.preprocessing import StandardScaler

**Create a StandardScaler() object called scaler.**

In [ ]:
scaler = StandardScaler()

**Fit scaler to the features.**

In [ ]:
scaler.fit(data.drop(['Class'], axis=1))

**Use the .transform() method to transform the features to a scaled version.**

In [ ]:
scaled_data = scaler.transform(data.drop(['Class'], axis=1))

**Convert the scaled features to a dataframe and check the head of this dataframe to make sure the scaling worked.**

In [ ]:
data_scaled = pd.DataFrame(data=scaled_data, columns=data.drop(['Class'], axis=1).columns)
data_scaled.head()

## Train Test Split

*Create two objects X and y which are the scaled feature values and labels respectively.*

In [ ]:
X = data_scaled

In [ ]:
y = data['Class']

*Use SciKit Learn to create training and testing sets of the data:*

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Tensorflow

In [ ]:
feat_col = []
for col in data_scaled.columns:
  feat_col.append(tf.feature_column.numeric_column(col))

*Create a list of feature column objects using tf.feature.numeric_column()*

In [ ]:
feat_col

*Create an object called classifier which is a DNNClassifier from learn. Set it to have 2 classes and a [10,20,10] hidden unit layer structure:*

In [ ]:
classifier = tf.estimator.DNNClassifier(hidden_units=[10,20,10], feature_columns=feat_col, n_classes=2)

*Now create a tf.estimator.pandas_input_fn that takes in your X_train, y_train, batch_size and set shuffle=True. You can play around with the batch_size parameter if you want, but let's start by setting it to 20 since our data isn't very big.*

In [ ]:
input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(X_train, y_train, batch_size=20, shuffle=True)

*Now train classifier to the input function. Use steps=500. You can play around with these values if you want!*

*Note: Ignore any warnings you get, they won't effect your output*

In [ ]:
classifier.train(input_fn, steps=500)

## Model Evaluation

*Create another pandas_input_fn that takes in the X_test data for x. Remember this one won't need any y_test info since we will be using this for the network to create its own predictions. Set shuffle=False since we don't need to shuffle for predictions.*

In [ ]:
pred_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(X_test, batch_size=len(X_test), shuffle=False)

*Use the predict method from the classifier model to create predictions from X_test*

In [ ]:
predictions = list(classifier.predict(pred_fn))

In [ ]:
predictions[0]

In [ ]:
results = []
for pred in predictions:
  results.append(pred['class_ids'][0])

*Now create a classification report and a Confusion Matrix. Does anything stand out to you?*

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(confusion_matrix(y_test, results))

In [ ]:
print(classification_report(y_test, results))

## Optional Comparison

*You should have noticed extremely accurate results from the DNN model. Let's compare this to a Random Forest Classifier for a reality check!*

*Use SciKit Learn to Create a Random Forest Classifier and compare the confusion matrix and classification report to the DNN model*

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)

In [ ]:
rfc.fit(X_train, y_train)

In [ ]:
rfc_pred = rfc.predict(X_test)

In [ ]:
print(classification_report(y_test, rfc_pred))

In [ ]:
print(confusion_matrix(y_test, rfc_pred))

*It should have also done very well, possibly perfect! Hopefully you have seen the power of DNN!*

# Cheers!